In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from google.colab import drive
drive.mount('/content/drive')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/cnn_dailymail/train.csv")

In [ ]:
train_data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [ ]:
train_data = train_data.rename(columns={"highlights":"target_text", "article":"source_text"})
train_data = train_data[['source_text', 'target_text']]

In [ ]:
train_data

,source_text,target_text
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...
287108,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [ ]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
train_data['source_text'] = "summarize: " + train_data['source_text']
train_data

,source_text,target_text
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...
...,...,...
287108,summarize: By . James Rush . Former first daug...,Chelsea Clinton said question of running for o...
287109,summarize: An apologetic Vanilla Ice has given...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,summarize: America's most lethal sniper claime...,America's most lethal sniper made comment in i...
287111,summarize: By . Sara Malm . PUBLISHED: . 12:19...,A swarm of more than one million has crossed b...


In [ ]:
pip install simplet5

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(train_data, test_size=0.2)
train_df.shape, test_df.shape

((229690, 2), (57423, 2))

In [ ]:
from simplet5 import SimpleT5
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-base")

model = SimpleT5()



INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
model.train(train_df=train_df[:1000],
            eval_df=test_df[:100],
            source_max_token_len=400,
            target_max_token_len=120,
            batch_size=8, max_epochs=2, use_gpu=True,
            early_stopping_patience_epochs=2)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved. New best score: 1.526


Validating: 0it [00:00, ?it/s]

In [ ]:
import torch

In [ ]:
filepath = "model.pt"

# save the model
torch.save(model, filepath)

In [ ]:
model1 = torch.load('model.pt', map_location=torch.device('cuda'))


In [ ]:
model1.device= torch.device("cuda")

In [ ]:
text_input = """A 38-year-old woman in the US, who was apprehended twice for allegedly trying to jump the
White House fence last week, has been arrested for scaling a fence at the Treasury Building.
Marci Anderson Wahl of Everett, Washington, was arrested after an alarm sounded at about 2:15 am yesterday
when she scaled a fence at the Treasury Building, next to the White House. Police said Wahl has told them
she was there to speak to US President Donald Trump, the CNN reported. She was charged with unlawful entry
and contempt of court. Wahl was first arrested on March 21 last week for trying to jump the White House fence.
Once in custody, it was determined that Wahl had been issued a "stay away" order for the White House complex
after the incident. On March 21, officers saw Wahl walking and staring at the White House complex before
discovering she had jumped a fence on the south side but got stuck. Officers found her hanging from the inside
of the fence by her shoelaces, which were "caught on top of the fence," according to a police report. She was
re-arrested on March 24 after officers saw her near Lafayette Park. She was released on her own recognisance
after a Saturday court appearance. In the incident, Wahl was charged with contempt of court in violation of a
stay away order. She pleaded not guilty last Saturday and was again released on her own recognisance."""


In [ ]:
text_to_summarize = """summarize: """ + text_input
predicted_output = model1.predict(text_to_summarize)

In [ ]:
predicted_output

['Marci Anderson Wahl of Everett, Washington, was apprehended twice last week. She is charged with unlawful entry and contempt of court.']

In [ ]:
text_input = """To,The Principal,Delhi Public School, Dwarka Sector 3, Dwarka, New Delhi
,Subject: Request to Dissolve the Current Unelected PTA and conduct Fresh Elections with Immediate Effect. Respected Madam,It is an undeniable fact that parents are the major stakeholders in the functioning of the school. I would like to bring to your attention that the Parents Teacher Association, in schools, plays a potential role in bringing parents and teacher together and to understand each other and cooperate in the task of development of young impressionable minds. The two agencies must make a concerted effort in evolving a conducive atmosphere in school for children to attain their optimum potential. We would like to bring to your notice that on 12th April’2010, D.O.E had issued Guidelines for Constitution of Parents Teachers Association (P.T.A) in Aided / Unaided Schools. Whether any School recognizes or not, Parents are major stakeholders in the functioning of a School. However, Parents of Delhi Public School, Dwarka understand that there were no elections held for section of P.T.A representatives. Our concerns about the existing PTA stem from several factors, including a lack of transparency in decision-making processes, limited parent involvement, and an inadequate representation of diverse perspectives. These issues have resulted in a perceived disconnect between the PTA and the Parents and Teachers, hindering effective collaboration and mutual understanding.
Moreover, it has been noticed that the Parent Rep of a particular class is promoted as the parent rep of the subsequent class in the next academic session without any consent, elections, or intimation to rest of the parents. The current practice of PTA had jeopardized the entire purpose for which it was constituted, most of the parents are not even aware who is the PTA of their ward’s class. There is no interaction between parents and PTA taking place which can strengthen the healthy communication amongst teachers and parents for overall development of ward and the school in totality.By initiating a fresh election of PTA members, we aim to provide an opportunity for all interested parents and teachers to actively participate in shaping the future of our school community. This process will enable the elections of PTA representatives who can foster open communication, encourage greater parental involvement, and work collaboratively with the school administration to address the evolving needs and concerns of students, parents, and teachers alike.We are also not sure if the any of the staff member of school are chosen as Parents Rep, if this is the case, it will lose the objective of basic purpose of constituting the Parents Teachers Association The staff members are bound to their Service Rules and have many constraints so many not project Parents point objectively in free and fair manner.Therefore, considering the above, we would like to request you to:
i.Dissolve the current unelected PTA with immediate effect. ii.Notify for conducting fresh and new fair and transparent P.T.A elections immediately in the current session before the given deadline. iii.School should notify the elections, personally informing all parents of the date and time through hard copy circulars with acknowledgment receipts for office use and SMS etc. iv.Eligible parents are to be encouraged to file the nominations and should be given enough time to file the nomination.v.One parent representative should not be allowed to continue for two consecutive terms. Since at present, there is a practice to continue the same person as a Parent representative for many years, which has seriously demeaned the main purpose of PTA to work in close coordination with teachers for the overall development of children.vi.One child One vote principle should be adopted. Only one vote for one parent may be verified with the Admission no. of the child.vii.Results may be published online without highlighting the names of parents who participated in elections. So that transparency be ensured.viii.The entire election process should be overlooked by a presiding officer, who can be a nominee from the DOE or any independent government official. ix.The complete elections process must be 100% transparent and elections, counting and declaration of results etc. should be held under the surveillance of a Video Camera. To ensure transparency and fairness, we recommend the formation of an independent election committee comprising both parents and teachers who are not currently associated with the PTA. This committee can oversee the election process, including the nomination of candidates, the dissemination of information, and the conduct of the actual voting.Requesting you to release the notification for free and fair election with immediate effect to ensure the principle of PTA body is exercised in true letter and spirit."""


In [ ]:
text_to_summarize = """summarize: """ + text_input
predicted_output = model1.predict(text_to_summarize)

Token indices sequence length is longer than the specified maximum sequence length for this model (1007 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
predicted_output

['Parents Teacher Association (PTA) is an important part of the functioning of a school. The current PTA has jeopardized the entire purpose for which it was formed.']

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/cnn_dailymail/test.csv")

In [ ]:
test_data = test_data.rename(columns={"highlights":"target_text", "article":"source_text"})


In [ ]:
test_data = test_data[['source_text', 'target_text']]

In [ ]:
text_summaries = []  # reference summary
for x in range(1, 101):
    summary = str(test_data['target_text'][x])
    summary = summary.replace("\n", "")
    print(summary)
    print("\n")
    text_summaries.append(summary)



Drunk teenage boy climbed into lion enclosure at zoo in west India .Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!'Fortunately he fell into a moat before reaching lions and was rescued .


Nottingham Forest are close to extending Dougie Freedman's contract .The Forest boss took over from former manager Stuart Pearce in February .Freedman has since lead the club to ninth in the Championship .


Fiorentina goalkeeper Neto has been linked with Liverpool and Arsenal .Neto joined Firoentina from Brazilian outfit Atletico Paranaense in 2011 .He is also wanted by PSG and Spanish clubs, according to his agent .CLICK HERE for the latest Liverpool news .


Tell-all interview with the reality TV star, 69, will air on Friday April 24 .It comes amid continuing speculation about his transition to a woman and following his involvement in a deadly car crash in February .The interview will also be one of Diane Sawyer's first appearances on television following the sudden death of h

In [ ]:
text_summaries[0]

"Drunk teenage boy climbed into lion enclosure at zoo in west India .Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!'Fortunately he fell into a moat before reaching lions and was rescued ."

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/cnn_dailymail/test.csv")

test_data = test_data.rename(columns={"highlights":"target_text", "article":"source_text"})

test_data = test_data[['source_text', 'target_text']]


In [ ]:
model_summaries = []
for x in range(1, 101):
  try:
    model_summary = model1.predict(test_data['source_text'][x])
    model_summaries.append(model_summary)
  except:
    continue


    #print(model_summaries[x])

In [ ]:
len(model_summaries)

100

In [ ]:
pip install rouge

In [ ]:
#summary_model

In [ ]:
model_summaries[0] #predicted

["Rahul Kumar, 17, clambered over enclosure fence at zoo in Ahmedabad. He had been sitting near the animals when he made a rush for them. Intoxicated teenager ran towards the lions shouting 'Today I kill a lion'"]

In [ ]:
text_summaries[0] #reference

"Drunk teenage boy climbed into lion enclosure at zoo in west India .Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!'Fortunately he fell into a moat before reaching lions and was rescued ."

In [ ]:
from rouge import Rouge

model_summaries = [str(summary) for summary in model_summaries]
text_summaries = [str(summary) for summary in text_summaries]

rouge = Rouge()
rouge.get_scores(model_summaries, text_summaries, avg=True, ignore_empty=True)

# from rouge import Rouge


# rouge = Rouge()
# rouge.get_scores(model_summaries, text_summaries, avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.3471658446985927,
  'p': 0.34232963887013135,
  'f': 0.3364772875321374},
 'rouge-2': {'r': 0.13841333242325118,
  'p': 0.13917400802609367,
  'f': 0.134372516107722},
 'rouge-l': {'r': 0.3288034480954962,
  'p': 0.3243130282707757,
  'f': 0.3187774115116153}}